In [76]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

from langchain.prompts import PromptTemplate

In [77]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [78]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [79]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1iT411q7GM
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/484220007107407896653ecb5f2122578ac2f3848cad97b7709?auth_key=1720686249-2427fab6587b4f40a93d368e526e3c50-0-a9711715810f3834a8e6cce42ba1d67d
Extracted segment: 15Y4y1S7Kb
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/654142091107406688224ec815f898b393f51d4bf087e552219?auth_key=1720686253-550457e08c854e25a2662b0a21d85210-0-f1f75d36f8e30ffd7414abb6ade3b985
Extracted segment: 1y24y1L7dZ
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/7818091071075718488fcd05d7f2bf5e7db1e705d9f8d70fa22?auth_key=1720686257-67006735c5ee47d7aaea9be0744f2a3d-0-1352f7f70ea04f9831fb35668b884acd
Extracted segment: 1Y84y1g7ah
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/6117549431075727729a9c113ae3840f

In [62]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [63]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

template_orig = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ### 板书一:     ### 板书二:       ### 讲解一:       ### 讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。### 板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$### 板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$### 讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。### 讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
            d: 对于所有题目尽可能用解释性的语言进行解答，不到万不得已不用公式直接求解，以最大限度上避免超纲的情况发生。
        13.板书不能超过10条，如果10条写不下，使用别的方法来解题。
"""

In [80]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
两个数相减的差是7.2，如果被减数增加8.6，减数减少2.7，则现在的差是多少？

### 板书一：
1. 原差：7.2
2. 被减数增加8.6，差增加8.6
3. 减数减少2.7，差增加2.7
4. 计算新的差：7.2 + 8.6 + 2.7

### 讲解一：
首先，我们知道两个数相减的差是7.2。接下来，题目告诉我们被减数增加了8.6，减数减少了2.7。我们需要理解的是，当被减数增加时，差也会增加相同的数值；当减数减少时，差也会增加相同的数值。

所以，第一步，我们将原来的差7.2加上被减数增加的8.6，得到新的差的一部分：
\[ 7.2 + 8.6 = 15.8 \]

第二步，我们再将这个结果加上减数减少的2.7，得到最终的差：
\[ 15.8 + 2.7 = 18.5 \]

因此，现在的差是18.5。

### 相似题一：
两个数相减的差是5.4，如果被减数增加6.3，减数减少1.8，则现在的差是多少？

### 板书一：
1. 原差：5.4
2. 被减数增加6.3，差增加6.3
3. 减数减少1.8，差增加1.8
4. 计算新的差：5.4 + 6.3 + 1.8

### 讲解一：
首先，我们知道两个数相减的差是5.4。接下来，题目告诉我们被减数增加了6.3，减数减少了1.8。我们需要理解的是，当被减数增加时，差也会增加相同的数值；当减数减少时，差也会增加相同的数值。

所以，第一步，我们将原来的差5.4加上被减数增加的6.3，得到新的差的一部分：
\[ 5.4 + 6.3 = 11.7 \]

第二步，我们再将这个结果加上减数减少的1.8，得到最终的差：
\[ 11.7 + 1.8 = 13.5 \]

因此，现在的差是13.5。executing
### 原题：
一个九位数各个数位上的数字之和是13，其中万位上的数字是一位上的两倍，问这个数最大是多少？

### 板书一：
1. 写出九个方框表示九位数
2. 确定万位和个位的关系
3. 确定万位和个位的具体数字
4. 计算剩余数位的数字

### 板书二：
1. 9个方框：_ _ _ _ _ _ _ _ _
2. 万位 = 2 × 个位
3. 万位 = 8，个位 = 4
4. 4 + 8 = 12，剩余数字和为1
5. 最大数：100000084

### 讲解

In [65]:
extract_question_explanations(raw_problem_answer)

In [66]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

In [67]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n两个数相减的差是7.2，如果被减数增加8.6，减数减少2.7，则现在的差是多少？\n\n讲解:\n首先我们要理解题目中的关键变化。被减数增加8.6，那么差也会增加8.6。减数减少2.7，那么差也会增加2.7。最初的差是7.2，所以新的差是7.2 + 8.6 + 2.7。我们来计算一下，7.2 + 8.6 = 15.8，再加上2.7，15.8 + 2.7 = 18.5。所以现在的差是18.5。\n\n题目:\n一个九位数各个数位上的数字之和是13，其中万位上的数字是一位上的两倍，问这个数最大是多少？\n\n讲解:\n我们采用的方法是一个九位数，把它写上九个方框。第一步，写上九个方框，帮助孩子快速地找到对应的数位。分好数位之后，来看一下万位上的数字。万位上的数字是一位上的两倍。考虑这个数要最大，数字的最高位要最大。我们来考虑一下它最大的时候可以填几。最大是填九，但九的两倍是18，不能填18，所以九是不行的。填八的话是16，填七的话是14，填六的话是12，填五的话是10，都不可以。只有填四可以，万位上是8。我们还知道数位上的数字之和是13，4和8加起来等于12，还差1。这个1填到最靠前的位置，其余的填0。这样，这个数最大是800000004。\n\n题目:\n6161 x 39 - 3939 x 61\n\n讲解:\n这道题目里有两个乘号和一个减号，满足乘法分配率的逆运算形式，但没有相同的数。我们需要将其转化为有相同数的形式。首先，我们将6161拆分为61 x 101。具体步骤如下：\n\n6161 = 61 x 100 + 61 = 61 x (100 + 1) = 61 x 101\n\n同样地，我们将3939拆分为39 x 101：\n\n3939 = 39 x 100 + 39 = 39 x (100 + 1) = 39 x 101\n\n将这些代入原题，我们得到：\n\n61 x 101 x 39 - 39 x 101 x 61\n\n观察减号两边的数字，发现它们完全相同，因此结果为零。\n\n'

In [73]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [74]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig, max_tokens=2000))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书[二三四五六七八九十]+:[\s\S]*?)*?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl, max_tokens=2000))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是代数中的加减法运算，解题关键点是理解被减数和减数的变化对差的影响。

### 板书一:
设原被减数为$x$，减数为$y$，
依题意得：
$x - y = 5.4$

### 板书二:
被减数增加6.3，减数减少1.8，
则新的被减数为$x + 6.3$，
新的减数为$y - 1.8$。

### 板书三:
新的差为：
$(x + 6.3) - (y - 1.8)$

### 板书四:
化简得：
$x + 6.3 - y + 1.8$

### 板书五:
代入$x - y = 5.4$，
得：
$5.4 + 6.3 + 1.8$

### 板书六:
计算得：
$5.4 + 6.3 + 1.8 = 13.5$

故答案是：13.5

### 讲解一:
首先设原被减数为$x$，减数为$y$，根据题意可以得到一个方程$x - y = 5.4$。

### 讲解二:
接着，根据题意，被减数增加6.3，减数减少1.8，因此新的被减数为$x + 6.3$，新的减数为$y - 1.8$。

### 讲解三:
新的差可以表示为$(x + 6.3) - (y - 1.8)$。

### 讲解四:
将新的差进行化简，得到$x + 6.3 - y + 1.8$。

### 讲解五:
将原方程$x - y = 5.4$代入化简后的表达式中，得到$5.4 + 6.3 + 1.8$。

### 讲解六:
最后，进行计算，得到$5.4 + 6.3 + 1.8 = 13.5$，故答案是13.5。executing
### 讲解一:
首先，我们需要设定一些变量来表示题目中的被减数和减数。设原被减数为$x$，减数为$y$。根据题意，两个数相减的差是5.4，所以我们可以写出方程：
\[ x - y = 5.4 \]
这一步是为了明确题目给出的已知条件，并用数学表达式表示出来。

### 讲解二:
接下来，我们根据题目中的变化来调整被减数和减数。题目说被减数增加6.3，减数减少1.8。因此，新的被减数可以表示为$x + 6.3$，新的减数可以表示为$y - 1.8$。这一步是为了将题目中的变化具体化，用数学表达式表示出来。

### 讲解三:
现在，我们需要计算新的差。新的差就是新的被减数减去新的减数。用数学表达式表示为：
\[ (x + 6.3) - (y

In [75]:
with open("compare_results_p9.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')